In [1]:
# rag_ollama_chroma.py
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import os
from pathlib import Path

In [2]:
root_path = Path(os.getcwd()).parent.parent
ollama_model = root_path / "ollama-service" / "models" / "ko_llama" / "llama-3-Korean-Bllossom-8B.Q4_K_M.gguf"
embeddings_path = root_path / "ollama-service" / "models" / "BGE-m3-ko"
print(ollama_model)
print(embeddings_path)

f:\project_test\solutions\chat\test1_1\ollama-service\models\ko_llama\llama-3-Korean-Bllossom-8B.Q4_K_M.gguf
f:\project_test\solutions\chat\test1_1\ollama-service\models\BGE-m3-ko


In [ ]:
# HuggingFace 임베딩 모델 로드 (경로를 문자열로 변환하여 오류 방지)
embedding_model = HuggingFaceEmbeddings(
    model_name=str(embeddings_path)
)
# llm = Ollama(model=str(ollama_model))

In [4]:
vectorstore = FAISS.load_local("vector_db/pcn_web2", embedding_model, allow_dangerous_deserialization=True)

In [5]:
# query_embedding = embedding_model.embed_query("피씨엔 회사 소개")

# # 벡터 유사도 검색 (상위 3개 결과 반환)
# results = vectorstore.similarity_search_by_vector(query_embedding, search_kwargs={"k": 20, "score_threshold": 0.75,})

In [6]:
# 검색기(Retriever) 설정 (개선)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})  # "score_threshold": 0.75
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [7]:
# # 벡터서치 결과 예시 출력
# from langchain_core.documents import Document

# docs = retriever.get_relevant_documents("PCN 회사 소개")
# docs

In [8]:

llm = ChatOllama(
    model=str("ko-llama-8B"),
    temperature=0.1,  # 약간의 다양성 부여
    base_url="http://localhost:11434",
    callback_manager=callback_manager,
    retriever=retriever  # retriever를 llm에 직접 설정
)

# 프롬프트를 더 명확하고 자연스럽게 개선
system_template = (
    '''
    당신은 (주)피씨엔(PCN)의 친절하고 유능한 AI 어시스턴트입니다.\n
    사용자의 질문에 대해 신뢰할 수 있는 정보를 바탕으로 정확하고 간결하게 답변하세요.\n
    주어진 정보를 바탕으로 답변하세요.\n
    **피씨엔**, 또는 **PCN**이라는 정보를 입력받으면 피씨엔(기술), PCN 회사 소개를 답변하세요.\n
    피씨엔, PCN 회사 소개는 아래 내용을 기반으로 답변하세요.\n
    피씨엔(PCN)은 기술 회사로 Bigdata, XR, AI, SI 등 다양한 서비스를 제공합니다.\n
    회사개요, 프로젝트(Project), 주요 솔루션, 조직규모, 주요 고객, 연혁 등 회사 정보를 답변하세요.\n
    프로젝트 설명 시, 프로젝트 이름, 프로젝트 설명, 프로젝트 결과 등을 1~2문장으로 간단하게 답변하세요.\n
    답변에서 회사명은 PCN으로 표기하세요.\n
    '''
)
human_template = (
    """
    주어진 질문은 사실에 근거하여 답변하세요.\n
    답변은 한글로 작성하세요.\n
    답변은 주어진 질문에 대해 단계별로 논리적으로 생각하여 답변해 주세요.\n
    질문: {question}
    """
)

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = prompt | llm

In [9]:
result1 = chain.invoke({"question": "피씨엔 회사 소개"})
print(result1)

피씨엔(PCN)은 기술 회사로서 다양한 분야에서 혁신적인 솔루션을 제공하는 기업입니다. 주요 서비스로는 빅데이터, 확장 현실(XR), 인공지능(AI), 시스템 통합(SI) 등이 있습니다.

### 회사 개요
피씨엔은 cutting-edge 기술을 활용하여 고객의 요구에 맞춘 솔루션을 개발하고 제공합니다. 이를 통해 다양한 산업 분야에서 혁신적인 변화를 이끌어내고 있습니다.

### 프로젝트
피씨엔은 여러 가지 프로젝트를 진행하고 있습니다. 예를 들어, '스마트 시티' 프로젝트는 도시의 효율성을 높이기 위해 빅데이터와 AI 기술을 활용한 솔루션을 제공합니다. 이 프로젝트는 교통 관리, 에너지 효율성, 공공 안전 등 다양한 측면에서 긍정적인 영향을 미쳤습니다.

### 주요 솔루션
1. **빅데이터 분석**: 대량의 데이터를 효과적으로 분석하여 고객에게 유용한 정보를 제공합니다.
2. **확장 현실(XR)**: 가상 현실(VR)과 증강 현실(AR)을 활용하여 교육, 마케팅, 의료 등 다양한 분야에서 혁신적인 경험을 제공합니다.
3. **인공지능(AI)**: 머신 러닝 기술을 적용하여 자동화와 예측 분석을 통해 고객의 비즈니스 효율성을 높입니다.
4. **시스템 통합(SI)**: IT 인프라를 통합하여 기업의 운영 효율성을 극대화합니다.

### 조직 규모
피씨엔은 다양한 분야의 전문가들로 구성된 강력한 팀을 보유하고 있습니다. 이를 통해 고객에게 최상의 서비스를 제공할 수 있습니다.

### 주요 고객
피씨엔의 주요 고객은 다양한 산업 분야에서 활동하는 기업들이며, 특히 금융, 제조, 유통, 교육 등이 포함됩니다.

### 연혁
피씨엔은 2010년에 설립되었으며, 이후 꾸준히 기술 혁신을 추구하며 성장해왔습니다. 주요 프로젝트와 솔루션의 성공적인 적용으로 인해 글로벌 시장에서도 주목받고 있습니다.

이러한 강점과 경험을 바탕으로 피씨엔은 앞으로도 지속적으로 발전하고 고객에게 최상의 서비스를 제공할 것입니다.content="피씨엔(PCN)은 기술 회사로서 다양한 분

In [11]:
from langchain.evaluation.qa import QAEvalChain

# 평가용 질문-정답 쌍 예시
eval_data = [
    {
        "query": "PCN의 주요 서비스는 무엇인가요?",
        "answer": "PCN은 빅데이터, XR, AI, SI 등 다양한 기술 서비스를 제공합니다."
    },
    {
        "query": "PCN의 설립 연도는 언제인가요?",
        "answer": "PCN은 2015년에 설립되었습니다."
    },
    {
        "query": "PCN의 주요 고객은 어떤 산업 분야에 있나요?",
        "answer": "PCN의 주요 고객은 금융, 제조, 유통, 교육 등 다양한 산업 분야에 있습니다."
    }
]


In [12]:
# Cell 11: 예측 생성 및 평가
# 체인에 질문을 넣어 답변 생성
predictions = []
for item in eval_data:
    try:
        result = chain.invoke({"question": item["query"]})
        predictions.append(str(result))
        print(f"질문 처리 완료: {item['query'][:30]}...")
    except Exception as e:
        print(f"질문 처리 실패: {item['query']} - 에러: {e}")
        predictions.append("")  # 빈 문자열로 채움

# 길이 확인
print(f"eval_data 길이: {len(eval_data)}")
print(f"predictions 길이: {len(predictions)}")

# predictions를 딕셔너리 형태로 변경
predictions_dict = []
for pred in predictions:
    predictions_dict.append({"result": pred})

print(f"predictions_dict 길이: {len(predictions_dict)}")

# 평가 체인 준비
eval_chain = QAEvalChain.from_llm(llm=llm)

# 평가 실행
graded_outputs = eval_chain.evaluate(
    examples=eval_data,
    predictions=predictions_dict,
    question_key="query",
    answer_key="answer"
)

# 평가 결과 출력
for i, (item, pred, grade) in enumerate(zip(eval_data, predictions, graded_outputs)):
    print(f"질문 {i+1}: {item['query']}")
    print(f"정답: {item['answer']}")
    print(f"모델 답변: {pred}")
    print(f"평가 결과: {grade['results'] if 'results' in grade else grade}")
    print("-" * 50)

PCN의 주요 서비스는 다음과 같습니다:

1. **Bigdata 서비스**: PCN은 대량 데이터를 수집, 저장, 분석하는 Bigdata 솔루션을 제공합니다. 이를 통해 고객은 데이터 기반 의사결정을 내릴 수 있으며, 비즈니스 운영 효율성을 높일 수 있습니다.

2. **XR(Extended Reality) 서비스**: PCN은 가상 현실(VR), 증강 현실(AR), 혼합 현실(MR) 기술을 활용한 XR 솔루션을 제공합니다. 이를 통해 고객은 혁신적인 교육, 마케팅, 엔터테인먼트 경험을 구현할 수 있습니다.

3. **AI(Artificial Intelligence) 서비스**: PCN은 인공지능 기술을 활용하여 고객의 비즈니스 프로세스를 자동화하고 최적화하는 AI 솔루션을 제공합니다. 이를 통해 고객은 자원 낭비를 줄이고, 효율성을 높일 수 있습니다.

4. **SI(System Integration) 서비스**: PCN은 다양한 시스템을 통합하는 SI 솔루션을 제공하여 고객의 정보시스템 운영 효율성을 극대화합니다. 이를 통해 고객은 종합적인 IT 관리와 유지보수를 받을 수 있습니다.

이러한 주요 서비스를 통해 PCN은 다양한 산업 분야에서 고객에게 혁신적인 기술 지원을 제공하고 있습니다.질문 처리 완료: PCN의 주요 서비스는 무엇인가요?...
PCN의 설립 연도는 현재 제공된 정보를 바탕으로 확인할 수 없습니다. 제공된 정보에 따르면 피씨엔(PCN)은 Bigdata, XR, AI, SI 등 다양한 서비스를 제공하는 기술 회사입니다. 그러나 PCN의 정확한 설립 연도를 알기 위해서는 추가적인 정보나 공식 문서가 필요합니다.질문 처리 완료: PCN의 설립 연도는 언제인가요?...
PCN의 주요 고객은 다양한 산업 분야에서 활동하고 있습니다. 특히, 기술 혁신과 데이터 분석이 중요한 분야에서 많은 수요가 있습니다. 다음은 PCN의 주요 고객이 속한 주요 산업 분야입니다:

1. **재무 및 금융**: PCN의 Bigdata 솔루션을 통해 금융 기관들은 거래

In [13]:
# 간단한 평가 (QAEvalChain 대신 직접 비교)
for i, (item, pred) in enumerate(zip(eval_data, predictions)):
    print(f"질문 {i+1}: {item['query']}")
    print(f"정답: {item['answer']}")
    print(f"모델 답변: {pred}")
    
    # 간단한 유사도 체크
    answer_words = set(item['answer'].lower().split())
    pred_words = set(pred.lower().split())
    common_words = answer_words.intersection(pred_words)
    similarity = len(common_words) / len(answer_words) if answer_words else 0
    
    print(f"유사도: {similarity:.2f}")
    print("-" * 50)

질문 1: PCN의 주요 서비스는 무엇인가요?
정답: PCN은 빅데이터, XR, AI, SI 등 다양한 기술 서비스를 제공합니다.
모델 답변: content='PCN의 주요 서비스는 다음과 같습니다:\n\n1. **Bigdata 서비스**: PCN은 대량 데이터를 수집, 저장, 분석하는 Bigdata 솔루션을 제공합니다. 이를 통해 고객은 데이터 기반 의사결정을 내릴 수 있으며, 비즈니스 운영 효율성을 높일 수 있습니다.\n\n2. **XR(Extended Reality) 서비스**: PCN은 가상 현실(VR), 증강 현실(AR), 혼합 현실(MR) 기술을 활용한 XR 솔루션을 제공합니다. 이를 통해 고객은 혁신적인 교육, 마케팅, 엔터테인먼트 경험을 구현할 수 있습니다.\n\n3. **AI(Artificial Intelligence) 서비스**: PCN은 인공지능 기술을 활용하여 고객의 비즈니스 프로세스를 자동화하고 최적화하는 AI 솔루션을 제공합니다. 이를 통해 고객은 자원 낭비를 줄이고, 효율성을 높일 수 있습니다.\n\n4. **SI(System Integration) 서비스**: PCN은 다양한 시스템을 통합하는 SI 솔루션을 제공하여 고객의 정보시스템 운영 효율성을 극대화합니다. 이를 통해 고객은 종합적인 IT 관리와 유지보수를 받을 수 있습니다.\n\n이러한 주요 서비스를 통해 PCN은 다양한 산업 분야에서 고객에게 혁신적인 기술 지원을 제공하고 있습니다.' additional_kwargs={} response_metadata={'model': 'ko-llama-8B', 'created_at': '2025-08-07T05:27:27.7310621Z', 'done': True, 'done_reason': 'stop', 'total_duration': 74842321500, 'load_duration': 67788200, 'prompt_eval_count': 284, 'prompt_eval_duration': 2539088800, 'eval_count

In [14]:
import evaluate

# 정답과 예측이 완전히 일치하면 1, 아니면 0으로 처리
references = []
preds = []

for item, pred in zip(eval_data, predictions):
    if item["answer"].strip() == pred.strip():
        references.append(1)
        preds.append(1)
    else:
        references.append(1)
        preds.append(0)

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

# evaluate 라이브러리는 zero_division 매개변수를 지원하지 않으므로 제거
accuracy = accuracy_metric.compute(references=references, predictions=preds)
precision = precision_metric.compute(references=references, predictions=preds, average="binary")
recall = recall_metric.compute(references=references, predictions=preds, average="binary")
f1 = f1_metric.compute(references=references, predictions=preds, average="binary")

print("=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===")
print(f"정확도(Accuracy): {accuracy['accuracy']:.2f}")
print(f"정밀도(Precision): {precision['precision']:.2f}")
print(f"재현율(Recall): {recall['recall']:.2f}")
print(f"F1 점수(F1 Score): {f1['f1']:.2f}")

=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===
정확도(Accuracy): 0.00
정밀도(Precision): 0.00
재현율(Recall): 0.00
F1 점수(F1 Score): 0.00


f:\project_test\solutions\chat\.venv_chat\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
import evaluate

# 정답과 예측이 완전히 일치하면 1, 아니면 0으로 처리
references = []
preds = []

for item, pred in zip(eval_data, predictions):
    if item["answer"].strip() == pred.strip():
        references.append(1)
        preds.append(1)
    else:
        references.append(1)
        preds.append(0)

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

# evaluate 라이브러리는 zero_division 매개변수를 지원하지 않으므로 제거
accuracy = accuracy_metric.compute(references=references, predictions=preds)
precision = precision_metric.compute(references=references, predictions=preds, average="binary")
recall = recall_metric.compute(references=references, predictions=preds, average="binary")
f1 = f1_metric.compute(references=references, predictions=preds, average="binary")

print("=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===")
print(f"정확도(Accuracy): {accuracy['accuracy']:.2f}")
print(f"정밀도(Precision): {precision['precision']:.2f}")
print(f"재현율(Recall): {recall['recall']:.2f}")
print(f"F1 점수(F1 Score): {f1['f1']:.2f}")

=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===
정확도(Accuracy): 0.00
정밀도(Precision): 0.00
재현율(Recall): 0.00
F1 점수(F1 Score): 0.00


f:\project_test\solutions\chat\.venv_chat\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [16]:
type(result1.content)

str

In [18]:
result_text = result1.content

In [19]:
reference="피씨엔(PCN)은 기술 회사로 Bigdata, XR, AI, SI 등 다양한 서비스를 제공합니다."

In [21]:

from langchain.chains import RetrievalQA

def rag_pipeline(input):
    # Extract retrieval context
    retrieved_docs = retriever.get_relevant_documents(input)
    context_texts = [doc.page_content for doc in retrieved_docs]

    # Generate response
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    result = qa_chain.invoke({"query": input})
    return result["result"], context_texts

In [23]:
from deepeval.test_case import LLMTestCase

input = '피씨엔 회사에 대해 소개해 주세요.'
actual_output, retrieved_contexts = rag_pipeline(input)

test_case = LLMTestCase(
    input=input,
    actual_output=actual_output,
    retrieval_context=retrieved_contexts,
    expected_output='피씨엔은 디지털 환경에서 고객 비즈니스 성공을 지원하는 Digital Value Service 기업입니다.'
)

f:\project_test\solutions\chat\.venv_chat\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
f:\project_test\solutions\chat\.venv_chat\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


죄송하지만, 저는 특정 회사의 정보를 제공할 수 없습니다. 하지만 일반적인 피씨엔(PCIe) 인터페이스에 대해 설명해 드릴 수 있습니다.

PCIe (Peripheral Component Interconnect Express)는 컴퓨터 하드웨어에서 데이터 전송을 위한 고속 인터페이스입니다. PCIe는 PCI (Peripheral Component Interconnect) 인터페이스의 후속 기술로, 더 빠른 속도와 더 많은 대역폭을 제공합니다. PCIe는 주로 그래픽 카드, 네트워크 카드, 저장 장치(예: SSD), 사운드 카드 등 다양한 외장 장치를 연결하는 데 사용됩니다.

PCIe 인터페이스는 여러 종류가 있으며, 그 중 가장 널리 사용되는 것은 다음과 같습니다:

1. **PCIe x1**: 가장 기본적인 형태로, 하나의 lane을 사용합니다.
2. **PCIe x4**: 네 개의 lane을 사용하여 더 빠른 데이터 전송이 가능합니다.
3. **PCIe x8**: 여덟 개의 lane을 사용하여 가장 높은 대역폭을 제공합니다.
4. **PCIe x16**: 주로 그래픽 카드에 사용되며, 넓은 bandwidth를 필요로 합니다.

PCIe는 슬롯 형태로 제공되며, 컴퓨터 메인보드에 설치됩니다. 각 슬롯에는 특정한 전압과 데이터 신호가 제공되어, 장치를 연결하고 데이터를 주고받을 수 있도록 합니다.

이와 같이 PCIe 인터페이스는 현대 컴퓨터 시스템에서 중요한 역할을 하며, 다양한 외장 장치의 빠른 데이터 전송을 가능하게 합니다.

In [3]:
import pytest
from deepeval import assert_test
from deepeval.dataset import EvaluationDataset, Golden
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

dataset = EvaluationDataset(goldens=[Golden(input="피씨엔 회사에 대해 소개해 주세요")])


In [13]:
@pytest.mark.parametrize(
    "test_case",
    dataset,
)
def test_customer_chatbot(test_case: LLMTestCase):
    answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5)
    assert_test(test_case, [answer_relevancy_metric])

In [14]:
for golden in dataset.goldens:
    test_case = LLMTestCase(
        input=golden.input,
        actual_output=chain.invoke({"question": golden.input})
    )
    dataset.add_test_case(test_case)

피씨엔(PCN)은 기술 회사로서 다양한 분야에서 혁신적인 솔루션을 제공하는 기업입니다. 주요 서비스로는 빅데이터, 확장 현실(XR), 인공지능(AI), 시스템 통합(SI) 등이 있습니다.

### 회사 개요
피씨엔은 cutting-edge 기술을 활용하여 고객의 요구에 맞춘 솔루션을 개발하고 제공합니다. 이를 통해 다양한 산업 분야에서 효율성을 높이고 혁신적인 변화를 이끌어냅니다.

### 주요 프로젝트
- **프로젝트 이름: Smart City Solution**
  - **프로젝트 설명:** 피씨엔은 스마트 시티 프로젝트를 통해 도시의 자원 관리, 교통 시스템, 에너지 효율성을 개선하는 솔루션을 개발했습니다.
  - **프로젝트 결과:** 이 프로젝트는 도시에 대한 데이터 분석과 예측 모델링을 통해 자원의 효율적인 사용을 가능하게 하였으며, 이는 도시의 환경 보호와 주민들의 삶의 질 향상에 기여했습니다.

### 주요 솔루션
- 빅데이터: 대량의 데이터를 수집하고 분석하여 고객에게 유용한 정보를 제공합니다.
- 확장 현실(XR): 가상과 증강 현실 기술을 활용하여 교육, 의료, 엔터테인먼트 분야에서의 혁신적인 경험을 제공합니다.
- 인공지능(AI): 머신 러닝과 딥러닝 알고리즘을 적용하여 자동화와 예측 분석을 지원합니다.
- 시스템 통합(SI): 다양한 시스템을 통합하여 효율적인 운영과 관리를 가능하게 합니다.

### 조직 규모
피씨엔은 기술 전문가, 데이터 과학자, 개발자 등 다양한 분야의 전문 인력을 보유하고 있습니다. 이를 통해 고객에게 최상의 서비스를 제공할 수 있도록 노력합니다.

### 주요 고객
피씨엔의 주요 고객은 다양한 산업 분야에서 활동하는 기업들입니다. 예를 들어, 제조업, 금융, 교육, 의료 등이 있으며, 이들은 피씨엔의 솔루션을 통해 경쟁력을 강화하고 혁신적인 변화를 추구합니다.

### 연혁
피씨엔은 2015년에 설립되었으며, 이후 다양한 프로젝트와 파트너십을 통해 기술적 성장을 이루어왔습니다. 2020년에는 AI 분야에서 세계적

In [10]:
from deepeval.models import OllamaModel
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval.metrics import (
    ContextualRelevancyMetric,
    ContextualRecallMetric,
    ContextualPrecisionMetric,
)

ollama_llm = OllamaModel(
    # 모델 이름, 베이스 URL, 온도(temperature) 설정
    # temperature는 생성되는 답변의 다양성(랜덤성)을 조절합니다. 0에 가까울수록 일관된 답변을, 1에 가까울수록 다양한 답변을 생성합니다.
    model=str("ko-llama-8B"),
    base_url="http://localhost:11434",
    temperature=0
)


relevancy = ContextualRelevancyMetric(model=ollama_llm)
recall = ContextualRecallMetric(model=ollama_llm)
precision = ContextualPrecisionMetric(model=ollama_llm)
answer_relevancy = AnswerRelevancyMetric(model=ollama_llm)

In [11]:
retrieval_context = retriever.get_relevant_documents("피씨엔 회사 소개")

C:\Users\PCN\AppData\Local\Temp\ipykernel_17852\4164084339.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieval_context = retriever.get_relevant_documents("피씨엔 회사 소개")
f:\project_test\solutions\chat\.venv_chat\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [12]:
doc_lis = [doc.page_content for doc in retrieval_context]

In [13]:
test_case = LLMTestCase(
    input="피씨엔 회사에 대해 소개해 주세요",
    actual_output=str(result1.content),
    retrieval_context=list(doc_lis),
    expected_output="피씨엔은 디지털 환경에서 고객 비즈니스 성공을 지원하는 Digital Value Service 기업입니다."
)

In [14]:
from deepeval.metrics import GEval

answer_correctness = GEval(
    model=ollama_llm,
    name="Answer Correctness",
    criteria="실제 출력의 'answer' 속성이 입력과 검색된 컨텍스트를 바탕으로 피씨엔에 대해 올바르고 완전하게 설명하는지 평가하세요. 답변이 올바르지 않거나 불완전하다면 점수를 감점하세요.",
    # LLMTestCaseParams가 정의되어 있지 않으므로 문자열로 대체
    evaluation_params=["input", "actual_output", "retrieval_context"]
)

citation_accuracy = GEval(
    model=ollama_llm,
    name="Citation Accuracy",
    criteria="실제 출력의 인용구가 입력과 검색된 컨텍스트를 바탕으로 올바르고 관련성이 있는지 평가하세요. 인용구가 올바르지 않거나 관련성이 없다면 점수를 감점하세요.",
    evaluation_params=["input", "actual_output", "retrieval_context"]
)

In [15]:
from deepeval.evaluate import evaluate, AsyncConfig

retriever_metrics = [relevancy, recall] # precision

result = evaluate(
    [test_case],
    retriever_metrics,
    async_config=AsyncConfig(run_async=False)
)
print(result.summary())


✨ You're running DeepEval's latest Contextual Relevancy Metric! (using ko-llama-8B (Ollama), strict=False, 
async_mode=False)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using ko-llama-8B (Ollama), strict=False, 
async_mode=False)...

Output()



Metrics Summary

  - ✅ Contextual Relevancy (score: 0.7075098814229249, threshold: 0.5, strict: False, evaluation model: ko-llama-8B (Ollama), reason: The input provided is too long and exceeds the maximum allowed length for a single message. Please break it down into smaller chunks or provide more context., error: None)
  - ❌ Contextual Recall (score: 0.0, threshold: 0.5, strict: False, evaluation model: ko-llama-8B (Ollama), reason: The score is 0.00 because the provided output lacks sufficient context to accurately support the expected response., error: None)

For test case:

  - input: 피씨엔 회사에 대해 소개해 주세요
  - actual output: 피씨엔(PCN)은 기술 회사로서 다양한 분야에서 혁신적인 솔루션을 제공하는 기업입니다. 주요 서비스로는 빅데이터, 확장 현실(XR), 인공지능(AI), 시스템 통합(SI) 등이 있습니다.

### 회사 개요
피씨엔은 cutting-edge 기술을 활용하여 고객의 요구에 맞춘 솔루션을 개발하고 제공합니다. 이를 통해 다양한 산업 분야에서 혁신적인 변화를 이끌어내고 있습니다.

### 프로젝트
피씨엔은 여러 가지 프로젝트를 진행하고 있습니다. 예를 들어, '스마트 시티' 프로젝트는 도시의 효율성을 높이기 위해 빅데이터와 AI 기술을 활용한 솔루션을 제공합니다. 이 프로젝트는 교통 관리, 에너지 효율성, 공공 안전 등 다양한 측면에서 긍정적인 영향

✓ Evaluation completed 🎉! (time taken: 6079.81s | token cost: 0.0 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

AttributeError: 'EvaluationResult' object has no attribute 'summary'

In [16]:
result.dict()

C:\Users\PCN\AppData\Local\Temp\ipykernel_17852\3026971379.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result.dict()


{'test_results': [{'name': 'test_case_0',
   'success': False,
   'metrics_data': [{'name': 'Contextual Relevancy',
     'threshold': 0.5,
     'success': True,
     'score': 0.7075098814229249,
     'reason': 'The input provided is too long and exceeds the maximum allowed length for a single message. Please break it down into smaller chunks or provide more context.',
     'strict_mode': False,
     'evaluation_model': 'ko-llama-8B (Ollama)',
     'error': None,
     'evaluation_cost': 0.0,
     'verbose_logs': 'Verdicts:\n[\n    {\n        "verdicts": [\n            {\n                "statement": "PCN, 2024\\ub144 \\uc8fc\\uc8fc\\uba85\\ubd80 \\ud3d0\\uc1c4\\uae30\\uac04 \\ubc0f \\uae30\\uc900\\uc77c \\uc124\\uc815 \\uacf5\\uace0",\n                "verdict": "yes",\n                "reason": null\n            },\n            {\n                "statement": "\\ud53c\\uc528\\uc5d4, \\u20182023 K-ICT \\uc704\\ud06c \\uc778 \\ubd80\\uc0b0\\u2019 \\ucc38\\uac00",\n                "verdic

In [ ]:
from deepeval import evaluate

generator_metrics = [answer_correctness, citation_accuracy]

evaluate(test_case, generator_metrics)

In [ ]:
# from deepeval import evaluate
# from deepeval.metrics import ContextualRecallMetric, ContextualRelevancyMetric


In [ ]:
answer_relevancy_metric = AnswerRelevancyMetric(model=ollama_llm)

# Evaluate
answer_relevancy_metric.measure(test_case)
print(answer_relevancy_metric.score)
print(answer_relevancy_metric.reason)

Output()

In [2]:
from deepeval.evaluate import evaluate, AsyncConfig

retriever_metrics = [relevancy, recall] # precision

result = evaluate(
    [test_case],
    retriever_metrics,
    async_config=AsyncConfig(run_async=False)
)
print(result.summary())


NameError: name 'relevancy' is not defined

In [ ]:
from deepeval.evaluate import evaluate

retriever_metrics = [relevancy, recall, precision]

# test_case 하나라도 리스트에 담아 호출
result = evaluate([test_case], retriever_metrics)
print(result.summary())


✨ You're running DeepEval's latest Contextual Relevancy Metric! (using ko-llama-8B (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using ko-llama-8B (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using ko-llama-8B (Ollama), strict=False, 
async_mode=True)...

Output()

In [ ]:
from deepeval import evaluate

retriever_metrics = [relevancy, recall, precision]

evaluate(test_case, retriever_metrics)

In [17]:
from deepeval import evaluate

evaluate(
    test_cases=test_case, 
    metrics=[answer_relevancy]
)

TypeError: 'LLMTestCase' object is not iterable

In [ ]:

# Evaluate
answer_relevancy.measure(test_case)
print(answer_relevancy.score)
print(answer_relevancy.reason)

Output()

In [31]:
# from deepeval import DeepEval
from deepeval import evaluate
from deepeval.metrics import ContextualRecall, Faithfulness

# 예시: 로컬 llama-cpp 모델 호출 함수
def rag_fn(query):
    return chain.invoke({"question": query})

evaluator = evaluate(rag_fn)
evaluator.add_test_case(
    name="간단 설명",
    query="피씨엔 회사에 대해 소개해 주세요",
    gold_answer="피씨엔은 디지털 환경에서 고객 비즈니스 성공을 지원하는 Digital Value Service 기업입니다.",
    metrics=[ContextualRecall(), Faithfulness()]
)
results = evaluator.run_all()
print(results.summary())


ImportError: cannot import name 'ContextualRecall' from 'deepeval.metrics' (f:\project_test\solutions\chat\.venv_chat\lib\site-packages\deepeval\metrics\__init__.py)

In [20]:
from deepeval import DeepEval
from deepeval.metrics import ContextualPrecision, AnswerRelevancy

# 평가 대상 RAG 함수 정의 (LangChain 예시)
def rag_fn(query):
    return chain.run(query)

# 테스트 케이스 등록
evaluator = DeepEval(rag_fn)
evaluator.add_test_case(
    name="피씨엔 회사 소개 FAQ 테스트",
    query="피씨엔 회사에 대해 소개해 주세요.",
    gold_answer="피씨엔은 디지털 환경에서 고객 비즈니스 성공을 지원하는 Digital Value Service 기업입니다.",
    metrics=[ContextualPrecision(), AnswerRelevancy()]
)
# 실행 및 리포트 출력
results = evaluator.run_all()
print(results.summary())


ImportError: cannot import name 'DeepEval' from 'deepeval' (f:\project_test\solutions\chat\.venv_chat\lib\site-packages\deepeval\__init__.py)

In [ ]:
import evaluate

# 정답과 예측이 완전히 일치하면 1, 아니면 0으로 처리
references = []
preds = []

for item, pred in zip(eval_data, predictions):
    if item["answer"].strip() == pred.strip():
        references.append(1)
        preds.append(1)
    else:
        references.append(1)
        preds.append(0)

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

# evaluate 라이브러리는 zero_division 매개변수를 지원하지 않으므로 제거
accuracy = accuracy_metric.compute(references=references, predictions=preds)
precision = precision_metric.compute(references=references, predictions=preds, average="binary")
recall = recall_metric.compute(references=references, predictions=preds, average="binary")
f1 = f1_metric.compute(references=references, predictions=preds, average="binary")

print("=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===")
print(f"정확도(Accuracy): {accuracy['accuracy']:.2f}")
print(f"정밀도(Precision): {precision['precision']:.2f}")
print(f"재현율(Recall): {recall['recall']:.2f}")
print(f"F1 점수(F1 Score): {f1['f1']:.2f}")

=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===
정확도(Accuracy): 0.00
정밀도(Precision): 0.00
재현율(Recall): 0.00
F1 점수(F1 Score): 0.00


f:\project_test\solutions\chat\.venv_chat\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
from deepeval import DeepEval
from deepeval.metrics import ContextualPrecision, AnswerRelevancy

# 평가 대상 RAG 함수 정의 (LangChain 예시)
def rag_fn(query):
    return chain.run(query)

# 테스트 케이스 등록
evaluator = DeepEval(rag_fn)
evaluator.add_test_case(
    name="피씨엔 회사 소개 FAQ 테스트",
    query="피씨엔 회사에 대해 소개해 주세요.",
    gold_answer="피씨엔은 디지털 환경에서 고객 비즈니스 성공을 지원하는 Digital Value Service 기업입니다.",
    metrics=[ContextualPrecision(), AnswerRelevancy()]
)
# 실행 및 리포트 출력
results = evaluator.run_all()
print(results.summary())


ImportError: cannot import name 'DeepEval' from 'deepeval' (f:\project_test\solutions\chat\.venv_chat\lib\site-packages\deepeval\__init__.py)

In [ ]:
result_test = evaluate(
    eval_data,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=llm_llama3,
    embeddings=embedding_model,
    run_config =RunConfig(timeout=600, max_retries=20, max_wait=50,log_tenacity=False),
)

AttributeError: 'ChatOllama' object has no attribute 'set_run_config'

In [ ]:
amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
eval_dataset = amnesty_qa["eval"].select(range(1,3))

FileNotFoundError: Couldn't find any data file at f:\project_test\solutions\chat\test1_1\chat-dev\src\explodinggradients\amnesty_qa. Couldn't find 'explodinggradients/amnesty_qa' on the Hugging Face Hub either: FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'C:\\Users\\PCN\\.cache\\huggingface\\hub\\datasets--explodinggradients--amnesty_qa'

In [ ]:
from ragas import SingleTurnSample
# from ragas.metrics import faithfulness, answer_relevancy
from ragas.metrics import Faithfulness, AnswerRelevancy
faithfulness = Faithfulness(llm=adapter)
answer_relevancy = AnswerRelevancy(llm=adapter)
# 평가에 사용할 샘플 데이터 생성
sample = SingleTurnSample(
    user_input="피씨엔 회사 소개",
    response=result_text,
    reference="피씨엔(PCN)은 기술 회사로 Bigdata, XR, AI, SI 등 다양한 서비스를 제공합니다."
)

# ragas의 각종 평가 지표 계산
faithfulness_score = faithfulness.single_turn_score(sample)
answer_relevancy_score = answer_relevancy.single_turn_score(sample)

print("=== RAGAS 기반 평가 결과 ===")
print(f"Faithfulness(정합성): {faithfulness_score:.2f}")
print(f"Answer Relevancy(응답 관련성): {answer_relevancy_score:.2f}")

AttributeError: 'ChatOllama' object has no attribute 'complete'

In [ ]:
from evaluate import load
import numpy as np

# 1) BERTScore
bs = load("bertscore")
out = bs.compute(
    predictions=[response],
    references=[[reference]],
    model_type="klue/bert-base"    # 한국어 모델
)
print("BERTScore F1:", out["f1"][0])

# 2) Sentence-Transformer 임베딩 유사도
from sentence_transformers import SentenceTransformer, util
model = embedding_model
emb1 = model.encode(response, convert_to_tensor=True)
emb2 = model.encode(reference, convert_to_tensor=True)
sim = util.cos_sim(emb1, emb2).item()
print("Cosine sim:", sim)


In [ ]:
import nest_asyncio
nest_asyncio.apply()

adapter = OllamaRagasAdapter(model_path="model.gguf", streaming=False)
faithfulness = Faithfulness(llm=adapter)
answer_relevancy = AnswerRelevancy(llm=adapter)

sample = SingleTurnSample(
    user_input="피씨엔 회사 소개",
    response="피씨엔(PCN)은 기술 회사로 Bigdata, XR, AI, SI 등 다양한 서비스를 제공합니다.",
    reference="피씨엔(PCN)은 IT 솔루션 및 플랫폼 개발 전문 기업입니다."
)

faith_score = faithfulness.single_turn_score(sample)
relevancy_score = answer_relevancy.single_turn_score(sample)
print("Faithfulness:", faith_score)
print("Relevancy:", relevancy_score)


AttributeError: 'dict' object has no attribute 'log_tenacity'

In [ ]:
from ragas import SingleTurnSample
# BleuScore는 기계 번역 등에서 예측된 텍스트와 참조(reference) 텍스트 간의 유사도를 평가하는 대표적인 지표입니다.
# BLEU(Bilingual Evaluation Understudy) 점수는 n-gram의 중복 정도를 기반으로 하며, 0에서 1 사이의 값을 가집니다.
# 값이 1에 가까울수록 예측 결과가 참조 문장과 유사함을 의미합니다.
from ragas.metrics import BleuScore
from ragas.metrics import faithfulness, answer_relevancy

# 평가에 사용할 샘플 데이터 생성
sample = SingleTurnSample(
    user_input="피씨엔 회사 소개",
    response=result1.content,
    reference="피씨엔(PCN)은 기술 회사로 Bigdata, XR, AI, SI 등 다양한 서비스를 제공합니다."
)

# ragas의 각종 평가 지표 계산
faithfulness_score = faithfulness.single_turn_score(sample)
answer_relevancy_score = answer_relevancy.single_turn_score(sample)
# ragas_precision_score = ragas_precision.single_turn_score(sample)
# ragas_recall_score = ragas_recall.single_turn_score(sample)

print("=== RAGAS 기반 평가 결과 ===")
print(f"Faithfulness(정합성): {faithfulness_score:.2f}")
print(f"Answer Relevancy(응답 관련성): {answer_relevancy_score:.2f}")
# print(f"Precision(정밀도): {ragas_precision_score:.2f}")
# print(f"Recall(재현율): {ragas_recall_score:.2f}")


# test_data = {
#     "user_input": "피씨엔 회사 소개",
#     "response": result1.content,
#     "reference": "피씨엔(PCN)은 기술 회사로 Bigdata, XR, AI, SI 등 다양한 서비스를 제공합니다."
# }
# metric = BleuScore()
# test_data = SingleTurnSample(**test_data)
# metric.single_turn_score(test_data)

AssertionError: LLM is not set

In [ ]:
import evaluate

# 정답과 예측이 완전히 일치하면 1, 아니면 0으로 처리
references = []
preds = []

for item, pred in zip(eval_data, predictions):
    if item["answer"].strip() == pred.strip():
        references.append(1)
        preds.append(1)
    else:
        references.append(1)
        preds.append(0)

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

# evaluate 라이브러리는 zero_division 매개변수를 지원하지 않으므로 제거
accuracy = accuracy_metric.compute(references=references, predictions=preds)
precision = precision_metric.compute(references=references, predictions=preds, average="binary")
recall = recall_metric.compute(references=references, predictions=preds, average="binary")
f1 = f1_metric.compute(references=references, predictions=preds, average="binary")

print("=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===")
print(f"정확도(Accuracy): {accuracy['accuracy']:.2f}")
print(f"정밀도(Precision): {precision['precision']:.2f}")
print(f"재현율(Recall): {recall['recall']:.2f}")
print(f"F1 점수(F1 Score): {f1['f1']:.2f}")

=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===
정확도(Accuracy): 0.00
정밀도(Precision): 0.00
재현율(Recall): 0.00
F1 점수(F1 Score): 0.00


f:\project_test\solutions\chat\.venv_chat\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
import evaluate

# 정답과 예측이 완전히 일치하면 1, 아니면 0으로 처리
references = []
preds = []

for item, pred in zip(eval_data, predictions):
    if item["answer"].strip() == pred.strip():
        references.append(1)
        preds.append(1)
    else:
        references.append(1)
        preds.append(0)

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

# evaluate 라이브러리는 zero_division 매개변수를 지원하지 않으므로 제거
accuracy = accuracy_metric.compute(references=references, predictions=preds)
precision = precision_metric.compute(references=references, predictions=preds, average="binary")
recall = recall_metric.compute(references=references, predictions=preds, average="binary")
f1 = f1_metric.compute(references=references, predictions=preds, average="binary")

print("=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===")
print(f"정확도(Accuracy): {accuracy['accuracy']:.2f}")
print(f"정밀도(Precision): {precision['precision']:.2f}")
print(f"재현율(Recall): {recall['recall']:.2f}")
print(f"F1 점수(F1 Score): {f1['f1']:.2f}")

=== 주요 성능지표 기반 RAG 평가 결과 (evaluate 사용) ===
정확도(Accuracy): 0.00
정밀도(Precision): 0.00
재현율(Recall): 0.00
F1 점수(F1 Score): 0.00


f:\project_test\solutions\chat\.venv_chat\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
 context_relevancy,